# Notebook to compute Overall Return Period

This notebook computes the return period for the action and observational triggers.

In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from src.datasources import codab, imerg, rsmc
from src.constants import *
from src import db_utils, utils

c:\Users\pauni\Desktop\Work\OCHA\GitHub\ds-aa-mdg-storms\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
adm1 = codab.load_codab(admin_level=1)

In [4]:
df_rsmc = rsmc.load_historical_forecast_distances()
distance_cols = [x for x in df_rsmc.columns if "_distance_km" in x]
df_rsmc["any_distance_km"] = df_rsmc[distance_cols].min(axis=1)

In [5]:
ibtracs_path = Path(AA_DATA_DIR) / "public" / "raw" / "glb" / "ibtracs"
points_path = Path(
    ibtracs_path
    / "IBTrACS.SI.list.v04r01.points/IBTrACS.SI.list.v04r01.points.shp"
)
gdf_points = gpd.read_file(points_path)
gdf_adm1_buffer = adm1.to_crs(epsg=mdg_epsg)
gdf_adm1_buffer["geometry"] = gdf_adm1_buffer.geometry.buffer(buffer * 1000)
gdf_adm1_buffer = gdf_adm1_buffer.to_crs(adm1.crs)
gdf_points_sel = gpd.sjoin(
    gdf_points, gdf_adm1_buffer, how="inner", predicate="intersects"
)
gdf_points_sel_ = gdf_points_sel.sort_values("ISO_TIME")

In [6]:
cyclone_db = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "mdg"
    / "cyclone_database"
    / "Synthèse dégâts Climatiques 1997 - 2024 (OCHA).xlsx"
)
cyclone_db = pd.read_excel(cyclone_db, sheet_name="BDD").dropna(how="all")
cyclone_db["Nom"] = cyclone_db["Nom"].str.upper()

## Readiness


In [7]:
scenario3_storms_readiness = df_rsmc[
    (df_rsmc["MG_distance_km"] <= 100)
    & (df_rsmc["max_wind_kt"] >= 90)
    & (df_rsmc["lt_hour"].between(72, 120))
]["name"].unique()

In [8]:
set(scenario3_storms_readiness)

{'BATSIRAI', 'BELNA', 'EMNATI', 'ENAWO', 'FREDDY', 'GAMANE', 'GIOVANNA'}

In [9]:
readiness_yr_len = df_rsmc["season"].nunique()
df_readiness_true = len(scenario3_storms_readiness)
print(
    f"The readiness trigger is met every {round(readiness_yr_len / df_readiness_true, 1)} years."
)

The readiness trigger is met every 2.0 years.


In [10]:
readiness_rate = df_readiness_true / readiness_yr_len
readiness_rate

0.5

## Action

In [11]:
action_yr_len = 2024 - 2001 + 1

In [12]:
action = set(
    gdf_points_sel_[
        (gdf_points_sel_["SEASON"] >= 2001)
        & (gdf_points_sel_["REU_WIND"] >= 90)
    ]["NAME"].unique()
)

In [13]:
action_rate = len(action) / action_yr_len
action_rate

0.3333333333333333

## Observational: Displaced Persons

In [14]:
obs_idp_yr_len = len(cyclone_db["Saison"].unique())

In [15]:
grouped_scenario = (
    cyclone_db.groupby("Nom")["Deplacees"].sum().apply(utils.classify_scenario)
)
grouped_df = pd.DataFrame({"Scenario": grouped_scenario}).reset_index()
obs_idp = set(grouped_df[grouped_df["Scenario"] == "Scenario 3"]["Nom"])

In [16]:
obs_idp_rate = len(obs_idp) / obs_idp_yr_len
obs_idp_rate

0.3103448275862069

## Observational: Rainfall

In [17]:
obs_rain_yr_len = 2024 - 2001 + 1

In [18]:
# From notebook 03.2_historical_rainfall
obs_rain = set(
    [
        "ENAWO",
        "MANOU",
        "INDLALA",
        "JADE",
        "ELIAKIM",
        "HUBERT",
        "GAFILO",
        "HEROLD",
    ]
)

In [19]:
obs_rain_rate = len(obs_rain) / obs_rain_yr_len
obs_rain_rate

0.3333333333333333

## Combined

### Combined probability (at least one activates)

#### Action and Observational IDP

In [24]:
common_cyclones = action.intersection(obs_idp)
total_cyclones = action.union(obs_idp)

probability_intersection = len(common_cyclones) / len(total_cyclones)

print(f"Common Cyclones: {common_cyclones}")
print(f"Probability of Intersection: {probability_intersection:.2f}")
print(
    f"Return Period: 1-in-{round(1 / (action_rate + obs_idp_rate - probability_intersection), 1)} yr"
)

Common Cyclones: {'ENAWO', 'INDLALA', 'GAFILO', 'IVAN'}
Probability of Intersection: 0.31
Return Period: 1-in-3.0 yr


#### Action, Observational IDP and Rainfall

In [30]:
common_cyclones_all = common_cyclones.intersection(obs_rain)
total_cyclones_all = common_cyclones.union(obs_rain)

probability_intersection_all = len(common_cyclones_all) / len(
    total_cyclones_all
)
combined_yr_len = max(action_yr_len, obs_idp_yr_len)
combined_rate = len(total_cyclones_all) / combined_yr_len
return_period = round(
    1 / (combined_rate + obs_rain_rate - probability_intersection_all), 1
)
print(f"Common Cyclones: {common_cyclones_all}")
print(f"Probability of Intersection: {probability_intersection_all:.2f}")
print(f"Return Period: 1-in-{return_period} yr")

Common Cyclones: {'ENAWO', 'GAFILO', 'INDLALA'}
Probability of Intersection: 0.33
Return Period: 1-in-3.2 yr
